# DDL

First connect to the database 'DS2':

In [ ]:
-- connection: host='localhost' dbname='ds2' user='ds2' 

----------------------------------------------------------------------------

## TP sur les DDL

* Créez une table nommée __'lp_cust'__ avec les champs suivants :
  - custId     identifiant unique ,
  - firstName  chaîne de 50 caractères,
  - lastName   chaîne de 50 caractères,
  - userName   chaîne de 50 caractères,

* Créer un type énuméré nommé __'lp_level'__ pouvant prendre les valeurs suivantes : 'gold', 'silver' ou 'bronze'.

* Créez une table nommée __'loyalty_program'__ avec les champs suivants :
  - custId     faisant référence à un identifiant d'utilisateur
  - lpLevel    un 'lp_level
  - star      un entier
 


In [ ]:
CREATE TYPE lp_level AS ENUM ('gold', 'silver', 'bronze');

CREATE TABLE lp_cust (
custId     SERIAL NOT NULL,
firstName  CHARACTER VARYING(50),
lastName   CHARACTER VARYING(50),
userName   CHARACTER VARYING(50),

);

CREATE TABLE loyalty_program (
custId     INTEGER NOT NULL,
lpLevel    lp_level,
star       INTEGER
);

* Si vous ne les avez pas intégrées à la conception des tables, ajouter les contraintes :
- pour identifier les users, les userNames doivent être uniques
- les consommateur ne peuvent avoir qu'un niveau dans le programme de fidélité
- la suppression d'un consommateur doit entrainer la suppression de son niveau dans le programme de fidélité

In [ ]:
ALTER TABLE lp_cust 
ADD UNIQUE(userName);

ALTER TABLE lp_cust
ADD CONSTRAINT lp_cust_pkey PRIMARY KEY (custId);         -- Nécessaire pour la définition de la clé étrangère ci-dessous

ALTER TABLE loyalty_program 
ADD CONSTRAINT fk_cust_hist_customerid FOREIGN KEY (custId) REFERENCES lp_cust(custId)
ON UPDATE CASCADE ON DELETE CASCADE;

* Lancez la commande suivante pour insérer des données dans la table 'lp_cust' :

In [ ]:
INSERT INTO lp_cust VALUES 
(1, 'Fred'   ,'Flinstone' ,'freddo'      ),
(2, 'John'   ,'Smith'     ,'jsmith'      ),
(3, 'Homer'  ,'Simpson'   ,'homey'       ),
(4, 'Homer'  ,'Brown'     ,'notsofamous' ),
(5, 'Ozzy'   ,'Ozzbourn'  ,'sabbath'     ),
(6, 'Homer'  ,'Gain'      ,'noplacelike' ),
(7, 'Jack'   ,'Brown'     ,'bigjack'     ),
(8, 'Morten' ,'Harket'    ,'aha'         ),
(9, 'John'   ,'Smith'     ,'jsmith1'     );

* Passez la commande suivante pour insérer des données dans la table 'loyalty_program' :

In [ ]:
INSERT INTO loyalty_program VALUES 
(1,         'gold'   ), 
(2,         'silver' ),
(3,         'silver' ),
(4,         'bronze' ),
(5,         'bronze' ),
(10,        'silver' ),
(10,        'bronze' ),
(11,        'gold'   );

* Modifiez la définition de la/des table(s) pour réaliser l'insertion.

In [ ]:
ALTER TABLE loyalty_program DROP CONSTRAINT IF EXISTS fk_cust_hist_customerid;

In [ ]:
INSERT INTO loyalty_program VALUES 
(1,         'gold'   ), 
(2,         'silver' ),
(3,         'silver' ),
(4,         'bronze' ),
(5,         'bronze' ),
(10,        'silver' ),
(10,        'bronze' ),
(11,        'gold'   );

* Le programme de fidélité, change, il ne contient plus de 'star'.

In [ ]:
ALTER TABLE loyalty_program DROP COLUMN star;

* Dans le modèle actuel du magasin de DVD, un client peut-il avoir plusieurs cartes de crédit ?

* Créez une table nommée 'cc_info' avec les infos relatives aux cartes de crédit 
  - une ligne est liée à un utilisateur
  - un utilisateur peut avoir plusieurs cartes de crédit

In [ ]:
DROP TABLE IF EXISTS cc_info;
CREATE TABLE cc_info (
cust_id INT,
cc_type INT4,
creditcard VARCHAR(50),
cc_exp VARCHAR(50),
CONSTRAINT fk_cc_info_customerid 
        FOREIGN KEY (cust_id) REFERENCES customers(customerid) 
        ON UPDATE CASCADE                                          -- suppression des moyens de paiement
        ON DELETE CASCADE                                          -- si le client est supprimé
)

* __Déplacez__ les données correspondantes dans la nouvelle table

In [ ]:
INSERT INTO cc_info (cust_id,cc_type,creditcard,cc_exp) 
SELECT customerid,creditcardtype,creditcard,creditcardexpiration
FROM customers;

ALTER TABLE customers
DROP COLUMN creditcardtype,
DROP COLUMN creditcard,
DROP COLUMN creditcardexpiration;

* Renommez la table 'cc_info' en 'credit_cards'

In [ ]:
ALTER TABLE cc_info RENAME TO credit_cards;